- 1단계 

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import time

# 손 모델 로딩
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

model = tf.keras.models.load_model('best_model_dnn_batch10.h5') 

# 웹캠 열기
cap = cv2.VideoCapture(0)

# 성공 신호 표시 여부를 저장하는 변수
success_signal = False
start_time = None
success_count1 = 0

# 시작 시간 기록
start_time_all = time.time()

# 반복 실행 (30초 동안)
while time.time() - start_time_all < 30:
    ret, frame = cap.read()
    if not ret:
        break

    # OpenCV를 사용하여 BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe를 사용하여 양손 감지
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks and len(results.multi_hand_landmarks) >= 2:
        # 좌표를 저장할 리스트 초기화
        all_x_coords_left = []
        all_y_coords_left = []
        all_x_coords_right = []
        all_y_coords_right = []
        
        for hand_landmarks in results.multi_hand_landmarks:
            # 감지된 손의 landmark를 사용하여 좌표 저장
            landmarks = hand_landmarks.landmark
            x_coords = [landmark.x for landmark in landmarks]
            y_coords = [landmark.y for landmark in landmarks]
            
            if landmarks[mp_hands.HandLandmark.WRIST].x < 0.5:
                all_x_coords_left.extend(x_coords)
                all_y_coords_left.extend(y_coords)
            else:
                all_x_coords_right.extend(x_coords)
                all_y_coords_right.extend(y_coords)

        if all_x_coords_left and all_y_coords_left and all_x_coords_right and all_y_coords_right:
            # Calculate bounding box encompassing both hands
            x_min_left = min(all_x_coords_left)
            y_min_left = min(all_y_coords_left)
            x_max_left = max(all_x_coords_left)
            y_max_left = max(all_y_coords_left)
            box_width_left = x_max_left - x_min_left
            box_height_left = y_max_left - y_min_left

            x_min_right = min(all_x_coords_right)
            y_min_right = min(all_y_coords_right)
            x_max_right = max(all_x_coords_right)
            y_max_right = max(all_y_coords_right)
            box_width_right = x_max_right - x_min_right
            box_height_right = y_max_right - y_min_right

            # Normalize coordinates by box width and height
            x_coords_normalized_left = [(x - x_min_left) * 100 / box_width_left for x in all_x_coords_left]
            y_coords_normalized_left = [(y - y_min_left) * 100 / box_height_left for y in all_y_coords_left]
            x_coords_normalized_right = [(x - x_min_right) * 100 / box_width_right for x in all_x_coords_right]
            y_coords_normalized_right = [(y - y_min_right) * 100 / box_height_right for y in all_y_coords_right]

            # 모델에 입력할 데이터 구성
            input_data_left = np.array([[x, y] for x, y in zip(x_coords_normalized_left, y_coords_normalized_left)])
            input_data_right = np.array([[x, y] for x, y in zip(x_coords_normalized_right, y_coords_normalized_right)])

            # 모델 예측
            pre1 = model.predict(np.expand_dims(input_data_left, axis=0))
            pre2 = model.predict(np.expand_dims(input_data_right, axis=0))

            # 분류 결과를 화면에 표시
            class_idx1 = np.argmax(pre1)
            class_idx2 = np.argmax(pre2)
            
            # putText에 넣을려면 문자열이여야되서 
            label1 = str(class_idx1)
            label2 = str(class_idx2)

            cv2.putText(frame, f"Left Hand: {label1}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, f"Right Hand: {label2}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

            if label1 == '1' and label2 == '1':
                if not success_signal:
                    start_time = time.time()  # 시작 시간 기록
                    success_signal = True  # 성공 신호를 활성화
                elif time.time() - start_time >= 1:  
                    cv2.putText(frame, f"Success!", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
                    output = 1
            else:
                if success_signal:  # 성공 텍스트가 꺼진 경우
                    success_count1 += 1  # 성공 카운트 증가
                    if success_count1 > 3:
                        success_count1 = 3
                    print(success_count1)
                    success_signal = False  # 성공 신호 비활성화


    cv2.imshow('Hand Classification', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 사용된 자원 해제
cap.release()
cv2.destroyAllWindows()
hands.close()

if success_count1 > 0:
    success_per1 = (success_count1 / 3) * 100  # 기준을 3으로 고정
else:
    success_per1 = 0  # success_count1이 0인 경우 성공률은 0으로 설정
    
print(f"성공률: {success_per1:.2f}")


1/1 [==============================] - 0s 33ms/step
1
1/1 [==============================] - 0s 33ms/step
2
1/1 [==============================] - 0s 32ms/step
3
1/1 [==============================] - 0s 32ms/step
3
1/1 [==============================] - 0s 24ms/step
3
1/1 [==============================] - 0s 24ms/step
3
1/1 [==============================] - 0s 32ms/step
3
1/1 [==============================] - 0s 23ms/step
3
1/1 [==============================] - 0s 40ms/step
3
1/1 [==============================] - 0s 24ms/step
성공률: 100.00


- 2단계

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import time

# 손 모델 로딩
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

model = tf.keras.models.load_model('best_model_dnn_batch10.h5') 

# 웹캠 열기
cap = cv2.VideoCapture(0)

# 성공 신호 표시 여부를 저장하는 변수
success_signal = False
start_time = None
success_count2 = 0
success_per2 = success_count2/5

# 시작 시간 기록
start_time_all = time.time()

# 반복 실행 (30초 동안)
while time.time() - start_time_all < 30:
    ret, frame = cap.read()
    if not ret:
        break

    # OpenCV를 사용하여 BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe를 사용하여 양손 감지
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks and len(results.multi_hand_landmarks) >= 2:
        # 좌표를 저장할 리스트 초기화
        all_x_coords_left = []
        all_y_coords_left = []
        all_x_coords_right = []
        all_y_coords_right = []
        
        for hand_landmarks in results.multi_hand_landmarks:
            # 감지된 손의 landmark를 사용하여 좌표 저장
            landmarks = hand_landmarks.landmark
            x_coords = [landmark.x for landmark in landmarks]
            y_coords = [landmark.y for landmark in landmarks]
            
            if landmarks[mp_hands.HandLandmark.WRIST].x < 0.5:
                all_x_coords_left.extend(x_coords)
                all_y_coords_left.extend(y_coords)
            else:
                all_x_coords_right.extend(x_coords)
                all_y_coords_right.extend(y_coords)

        if all_x_coords_left and all_y_coords_left and all_x_coords_right and all_y_coords_right:
            # Calculate bounding box encompassing both hands
            x_min_left = min(all_x_coords_left)
            y_min_left = min(all_y_coords_left)
            x_max_left = max(all_x_coords_left)
            y_max_left = max(all_y_coords_left)
            box_width_left = x_max_left - x_min_left
            box_height_left = y_max_left - y_min_left

            x_min_right = min(all_x_coords_right)
            y_min_right = min(all_y_coords_right)
            x_max_right = max(all_x_coords_right)
            y_max_right = max(all_y_coords_right)
            box_width_right = x_max_right - x_min_right
            box_height_right = y_max_right - y_min_right

            # Normalize coordinates by box width and height
            x_coords_normalized_left = [(x - x_min_left) * 100 / box_width_left for x in all_x_coords_left]
            y_coords_normalized_left = [(y - y_min_left) * 100 / box_height_left for y in all_y_coords_left]
            x_coords_normalized_right = [(x - x_min_right) * 100 / box_width_right for x in all_x_coords_right]
            y_coords_normalized_right = [(y - y_min_right) * 100 / box_height_right for y in all_y_coords_right]

            # 모델에 입력할 데이터 구성
            input_data_left = np.array([[x, y] for x, y in zip(x_coords_normalized_left, y_coords_normalized_left)])
            input_data_right = np.array([[x, y] for x, y in zip(x_coords_normalized_right, y_coords_normalized_right)])

            # 모델 예측
            pre1 = model.predict(np.expand_dims(input_data_left, axis=0))
            pre2 = model.predict(np.expand_dims(input_data_right, axis=0))

            # 분류 결과를 화면에 표시
            class_idx1 = np.argmax(pre1)
            class_idx2 = np.argmax(pre2)
            
            # putText에 넣을려면 문자열이여야되서 
            label1 = str(class_idx1)
            label2 = str(class_idx2)

            cv2.putText(frame, f"Left Hand: {label1}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, f"Right Hand: {label2}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

            if (label1 == '0' and label2 == '1') or (label1 == '1' and label2 == '0'):
                if not success_signal:
                    start_time = time.time()  # 시작 시간 기록
                    success_signal = True  # 성공 신호를 활성화
                elif time.time() - start_time >= 1:  # 1초 동안 연속적으로 1인 경우
                    cv2.putText(frame, f"Success!", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
                    output = 1
            else:
                if success_signal:  # 성공 텍스트가 꺼진 경우
                    success_count2 += 1  # 성공 카운트 증가
                    print(success_count2)
                    if success_count1 > 3:
                        success_count1 = 3
                    success_signal = False  # 성공 신호 비활성화


    cv2.imshow('Hand Classification', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 사용된 자원 해제
cap.release()
cv2.destroyAllWindows()
hands.close()


if success_count1 > 0:
    success_per1 = (success_count1 / 3) * 100  # 기준을 3으로 고정
else:
    success_per1 = 0  # success_count1이 0인 경우 성공률은 0으로 설정
    
print(f"성공률: {success_per1:.2f}")

1/1 [==============================] - 0s 32ms/step
1
1/1 [==============================] - 0s 25ms/step
2
1/1 [==============================] - 0s 24ms/step
3
1/1 [==============================] - 0s 25ms/step
4
1/1 [==============================] - 0s 24ms/step
5
1/1 [==============================] - 0s 32ms/step
6
1/1 [==============================] - 0s 24ms/step
7
1/1 [==============================] - 0s 32ms/step
8
1/1 [==============================] - 0s 24ms/step
9
1/1 [==============================] - 0s 24ms/step
10
1/1 [==============================] - 0s 24ms/step
11
1/1 [==============================] - 0s 24ms/step
성공률: 100.00


- 3단계

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import time

# 손 모델 로딩
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

model = tf.keras.models.load_model('best_model_dnn_batch10.h5') 

# 웹캠 열기
cap = cv2.VideoCapture(0)

# 성공 신호 표시 여부를 저장하는 변수
success_signal = False
start_time = None
success_count3 = 0


# 시작 시간 기록
start_time_all = time.time()

# 반복 실행 (30초 동안)
while time.time() - start_time_all < 60:
    ret, frame = cap.read()
    if not ret:
        break

    # OpenCV를 사용하여 BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe를 사용하여 양손 감지
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks and len(results.multi_hand_landmarks) >= 2:
        # 좌표를 저장할 리스트 초기화
        all_x_coords_left = []
        all_y_coords_left = []
        all_x_coords_right = []
        all_y_coords_right = []
        
        for hand_landmarks in results.multi_hand_landmarks:
            # 감지된 손의 landmark를 사용하여 좌표 저장
            landmarks = hand_landmarks.landmark
            x_coords = [landmark.x for landmark in landmarks]
            y_coords = [landmark.y for landmark in landmarks]
            
            if landmarks[mp_hands.HandLandmark.WRIST].x < 0.5:
                all_x_coords_left.extend(x_coords)
                all_y_coords_left.extend(y_coords)
            else:
                all_x_coords_right.extend(x_coords)
                all_y_coords_right.extend(y_coords)

        if all_x_coords_left and all_y_coords_left and all_x_coords_right and all_y_coords_right:
            # Calculate bounding box encompassing both hands
            x_min_left = min(all_x_coords_left)
            y_min_left = min(all_y_coords_left)
            x_max_left = max(all_x_coords_left)
            y_max_left = max(all_y_coords_left)
            box_width_left = x_max_left - x_min_left
            box_height_left = y_max_left - y_min_left

            x_min_right = min(all_x_coords_right)
            y_min_right = min(all_y_coords_right)
            x_max_right = max(all_x_coords_right)
            y_max_right = max(all_y_coords_right)
            box_width_right = x_max_right - x_min_right
            box_height_right = y_max_right - y_min_right

            # Normalize coordinates by box width and height
            x_coords_normalized_left = [(x - x_min_left) * 100 / box_width_left for x in all_x_coords_left]
            y_coords_normalized_left = [(y - y_min_left) * 100 / box_height_left for y in all_y_coords_left]
            x_coords_normalized_right = [(x - x_min_right) * 100 / box_width_right for x in all_x_coords_right]
            y_coords_normalized_right = [(y - y_min_right) * 100 / box_height_right for y in all_y_coords_right]

            # 모델에 입력할 데이터 구성
            input_data_left = np.array([[x, y] for x, y in zip(x_coords_normalized_left, y_coords_normalized_left)])
            input_data_right = np.array([[x, y] for x, y in zip(x_coords_normalized_right, y_coords_normalized_right)])

            # 모델 예측
            pre1 = model.predict(np.expand_dims(input_data_left, axis=0))
            pre2 = model.predict(np.expand_dims(input_data_right, axis=0))

            # 분류 결과를 화면에 표시
            class_idx1 = np.argmax(pre1)
            class_idx2 = np.argmax(pre2)
            
            # putText에 넣을려면 문자열이여야되서 
            label1 = str(class_idx1)
            label2 = str(class_idx2)
        

            cv2.putText(frame, f"Left Hand: {label1}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, f"Right Hand: {label2}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            
            if (label1 == '2' or label1 == '3') and (label2 == '2' or label2 == '3'):
                if not success_signal:
                    start_time = time.time()  # 시작 시간 기록
                    success_signal = True  # 성공 신호를 활성화
                elif time.time() - start_time >= 2:  # 1초 동안 연속적으로 1인 경우
                    cv2.putText(frame, f"Success!", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
                    output = 1
            elif (label1 == '4' or label1 == '5') and (label2 == '4' or label2 == '5'):
                if not success_signal:
                    start_time = time.time()  # 시작 시간 기록
                    success_signal = True  # 성공 신호를 활성화
                elif time.time() - start_time >= 2:  # 1초 동안 연속적으로 1인 경우
                    cv2.putText(frame, f"Success!", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
                    output = 1
            elif label1 == '0' and label2 == '0':
                if not success_signal:
                    start_time = time.time()  # 시작 시간 기록
                    success_signal = True  # 성공 신호를 활성화
                elif time.time() - start_time >= 2:  # 1초 동안 연속적으로 1인 경우
                    cv2.putText(frame, f"Success!", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
                    output = 1
            else:
                if success_signal:  # 성공 텍스트가 꺼진 경우
                    success_count3 += 1  # 성공 카운트 증가
                    print(success_count3)
                    success_signal = False  # 성공 신호 비활성화



    cv2.imshow('Hand Classification', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 사용된 자원 해제
cap.release()
cv2.destroyAllWindows()
hands.close()


1/1 [==============================] - 0s 33ms/step
1
1/1 [==============================] - 0s 27ms/step
2
1/1 [==============================] - 0s 24ms/step
3
1/1 [==============================] - 0s 32ms/step
4
1/1 [==============================] - 0s 24ms/step
5
1/1 [==============================] - 0s 24ms/step
6
1/1 [==============================] - 0s 33ms/step
7
1/1 [==============================] - 0s 32ms/step
8
1/1 [==============================] - 0s 31ms/step
9
1/1 [==============================] - 0s 23ms/step
10
1/1 [==============================] - 0s 25ms/step
11
1/1 [==============================] - 0s 33ms/step
12
1/1 [==============================] - 0s 33ms/step
13
1/1 [==============================] - 0s 32ms/step
14
1/1 [==============================] - 0s 32ms/step
15
1/1 [==============================] - 0s 24ms/step
16
1/1 [==============================] - 0s 30ms/step
17
1/1 [==============================] - 0s 24ms/step
18
1/1 [==============

- 1단계 (실시간 데이터 정확도 확인) 

In [9]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import time

# Load the hand model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Load the trained model
model = tf.keras.models.load_model('best_model_dnn_batch10.h5')

# Variables for success signal tracking
success_signal = False
start_time = None
success_count1 = 0

# Data accumulation variables
X_real_time = []  # Store real-time data
y_real_time = []  # Store corresponding labels

# Start time
start_time_all = time.time()

# Open webcam
cap = cv2.VideoCapture(0)

# Real-time data collection loop
while time.time() - start_time_all < 30:  # Run for 30 seconds, adjust as needed
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect hands using Mediapipe
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks and len(results.multi_hand_landmarks) >= 2:
        all_x_coords_left = []
        all_y_coords_left = []
        all_x_coords_right = []
        all_y_coords_right = []

        for hand_landmarks in results.multi_hand_landmarks:
            # Get coordinates of all landmarks
            landmarks = hand_landmarks.landmark
            x_coords = [landmark.x for landmark in landmarks]
            y_coords = [landmark.y for landmark in landmarks]

            # Determine which hand the landmarks belong to based on the position of the wrist landmark
            if landmarks[mp_hands.HandLandmark.WRIST].x < 0.5:
                all_x_coords_left.extend(x_coords)
                all_y_coords_left.extend(y_coords)
            else:
                all_x_coords_right.extend(x_coords)
                all_y_coords_right.extend(y_coords)

        if all_x_coords_left and all_y_coords_left and all_x_coords_right and all_y_coords_right:
            # Calculate bounding box encompassing both hands
            x_min_left = min(all_x_coords_left)
            y_min_left = min(all_y_coords_left)
            x_max_left = max(all_x_coords_left)
            y_max_left = max(all_y_coords_left)
            box_width_left = x_max_left - x_min_left
            box_height_left = y_max_left - y_min_left

            x_min_right = min(all_x_coords_right)
            y_min_right = min(all_y_coords_right)
            x_max_right = max(all_x_coords_right)
            y_max_right = max(all_y_coords_right)
            box_width_right = x_max_right - x_min_right
            box_height_right = y_max_right - y_min_right

            # Normalize coordinates by box width and height
            x_coords_normalized_left = [(x - x_min_left) * 100 / box_width_left for x in all_x_coords_left]
            y_coords_normalized_left = [(y - y_min_left) * 100 / box_height_left for y in all_y_coords_left]
            x_coords_normalized_right = [(x - x_min_right) * 100 / box_width_right for x in all_x_coords_right]
            y_coords_normalized_right = [(y - y_min_right) * 100 / box_height_right for y in all_y_coords_right]

            # Construct input data for the model
            input_data_left = np.array([[x, y] for x, y in zip(x_coords_normalized_left, y_coords_normalized_left)])
            input_data_right = np.array([[x, y] for x, y in zip(x_coords_normalized_right, y_coords_normalized_right)])

            # Append data to real-time arrays
            X_real_time.append(input_data_left)
            X_real_time.append(input_data_right)

            # Perform model prediction for each hand
            pred_left = model.predict(np.expand_dims(input_data_left, axis=0))
            pred_right = model.predict(np.expand_dims(input_data_right, axis=0))

            # Store predictions as labels
            y_real_time.append(np.argmax(pred_left))
            y_real_time.append(np.argmax(pred_right))

            # Display classification results on the screen
            class_idx1 = np.argmax(pred_left)
            class_idx2 = np.argmax(pred_right)

            label1 = str(class_idx1)
            label2 = str(class_idx2)

            cv2.putText(frame, f"Left Hand: {label1}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, f"Right Hand: {label2}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

            # Check for successful detection of a specific gesture
            if label1 == '1' and label2 == '1':
                if not success_signal:
                    start_time = time.time()
                    success_signal = True
                elif time.time() - start_time >= 1:
                    cv2.putText(frame, f"Success!", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
                    output = 1
            else:
                if success_signal:
                    success_count1 += 1
                    if success_count1 > 3:
                        success_count1 = 3
                    print(success_count1)
                    success_signal = False

    cv2.imshow('Hand Classification', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Convert real-time data to numpy arrays
X_real_time = np.array(X_real_time)
y_real_time = np.array(y_real_time)

# Evaluate the model on real-time data
loss, accuracy = model.evaluate(X_real_time, y_real_time)
print("Real-time data loss:", loss)
print("Real-time data accuracy:", accuracy)

# Release resources
cap.release()
cv2.destroyAllWindows()
hands.close()

if success_count1 > 0:
    success_per1 = (success_count1 / 3) * 100
else:
    success_per1 = 0
    
print(f"Success rate: {success_per1:.2f}%")



1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 31ms/step
2
1/1 [==============================] - 0s 24ms/step
3
1/1 [==============================] - 0s 8ms/step
3
1/1 [==============================] - 0s 63ms/step
3
1/1 [==============================] - 0s 63ms/step
3
1/1 [==============================] - 0s 60ms/step
3
1/1 [==============================] - 0s 48ms/step
3
1/1 [==============================] - 0s 62ms/step
3
1/1 [==============================] - 0s 50ms/step
3
1/1 [==============================] - 0s 72ms/step
3
6/6 [==============================] - 0s 5ms/step - loss: 0.0513 - accuracy: 1.0000
Real-time data loss: 0.05132707953453064
Real-time data accuracy: 1.0
Success rate: 100.00%
